In [1]:
import os 
os.chdir("nmt_training")

In [2]:
import datasets 
import transformers

from nmt_clean.config import config
from nmt_clean.config_m2m_mul_en import config as configm2m
from nmt_clean.load_data import load_training_data, load_validation_data, load_testing_data
from nmt_clean.metrics import compute_sacreBLEU as compute_metrics
from nmt_clean.preprocess import Many2ManyProcessor

from transformers import EarlyStoppingCallback

tokenizer = transformers.AutoTokenizer.from_pretrained(config["model_checkpoint"])
main_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(config['model_checkpoint'])
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, model = main_model) 



Evaluating every 30 training steps.


In [3]:
#compute metrics eval_lang to eval_pairs
#set the tgt tokens in the source

In [4]:
#tokenizer.lang_code_to_id = {}
#largest_token = len(tokenizer)
#for i, code in enumerate(["ach", "lgg", "lug", "nyn", "teo", "en"]):
#    tokenizer.lang_code_to_id[code] = i + largest_token
    #tokenizer.fairseq_ids_to_tokens[i + largest_token] = code

metric = datasets.load_metric('sacrebleu')
processor = Many2ManyProcessor()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  import sys


In [5]:


#paired_datasets = processor.dataset_from_folders_m2m(configm2m["training_subset_paths"])
#tokenizer = AutoTokenizer.from_pretrained(config["model_checkpoint"])

#paired_datasets = datasets.concatenate_datasets(paired_datasets)
#paired_datasets = paired_datasets.shuffle(seed=42)
#paired_datasets = paired_datasets.flatten_indices()  # rewrite the shuffled dataset on disk as contiguous chunks of data

#training_tokens = processor.iterative_preprocess([paired_datasets],tokenizer )



#validation_cutoff = 50

#os.environ["WANDB_API_KEY"] = secret_value_0
#wandb.init(project=config['wandb_project'],entity=config["wandb_entity"], config=config)



In [6]:
valid_datasets = processor.dataset_from_folders_m2m(configm2m["validation_subset_paths"])


100%|██████████| 6/6 [00:00<00:00, 13.03it/s]


In [7]:
4126*30

123780

In [8]:
eval_preds = []
for i in range(len(valid_datasets)):
    eval_preds.append(valid_datasets[i].src_language + "_" + valid_datasets[i].tgt_language)


In [9]:
def shrink_validation(dataset_to_shrink, eval_pairs, subset_len = 4126, samples_per_language = 100, pair_needed = "en"):
    subsets = []
    shrunk_eval_pairs = [] 
    assert subset_len > samples_per_language

    for language_pair, beginning_idx in zip(eval_pairs, range(0,len(dataset_to_shrink), subset_len)):
        if pair_needed not in language_pair:
            continue
        subset = datasets.Dataset.from_dict(dataset_to_shrink[beginning_idx:beginning_idx+samples_per_language])
        subsets.append(subset)
        shrunk_eval_pairs.append(language_pair)
    return subsets, shrunk_eval_pairs



In [10]:
processed_dataset = datasets.load_dataset("mekaneeky/salt_m2m_ready")

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
shrunk_validation, shrunk_eval_preds = shrink_validation(processed_dataset["validation"], eval_preds)

In [12]:
config['train_settings'].max_steps = 300

In [ ]:
main_model.config.use_cache = False

trainer = transformers.Seq2SeqTrainer(
    main_model,
    config['train_settings'],
    train_dataset = processed_dataset["train"],
    eval_dataset = shrunk_validation,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = lambda x: compute_metrics(
        x, eval_preds , config['validation_samples_per_language'], tokenizer,metric ),
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)],
)
#trainer.config = config['train_settings']
#trainer.config.max_length = config["max_input_length"] #FIXME issue PR to the repo
#trainer.config.num_beams = 5 #FIXME issue PR to the repo

train_result = trainer.train()
metrics = trainer.evaluate()
print(metrics)
trainer.save_model()

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3683940
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2400
  Gradient Accumulation steps = 2400
  Total optimization steps = 300
  Number of trainable parameters = 76994560
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: azawahry (sunbird). Use `wandb login --relogin` to force relogin


In [ ]:
val_tokens